In [1]:
import warnings
warnings.filterwarnings("ignore")
## setting up the language model
import os
from dotenv import load_dotenv
from langchain_together import ChatTogether
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain import LLMChain

## load the environment variables
load_dotenv()

True

In [2]:
llm = ChatTogether(api_key=os.getenv("TOGETHER_AI_API_KEY"),temperature=0.0, model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")

In [3]:
from langchain.document_loaders import PyPDFLoader # PDF loader from langchain

# Load PDF
loaders = [
    PyPDFLoader("./Gautam_Naik_CV.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [4]:
len(docs)

2

In [5]:
docs[0].page_content

"Gautam\nNaik\nS e n i o r\nS o f t w a r e\nE n g i n e e r\nGoa-India\n•\n+91\n9421873737\n•\ngautamnaik1994@gmail.com\n•\nlink edin.com/in/gautamnaik1994\n•\ngautamnaik.com\n●\nS e n i o r\nS o f t w a r e\nE n g i n e e r\nw i t h\no v e r\n8\ny e a r s\no f\ne x p e r i e n c e\nd e v e l o p i n g\nf r o n t e n d\ns y s t e m s\na n d\nr e s p o n s i v e\nw e b s i t e s\nu s i n g\nR e a c t J S ,\nN e x t J S ,\nT y p e s c r i p t ,\nR e d u x ,\nJ a v a S c r i p t ,\nE S 6 ,\na n d\nC S S .\n●\nP r o v e n\nt r a c k\nr e c o r d\no f\nl e a d i n g\na\nt e a m ,\na n d\nc o l l a b o r a t i n g\nw i t h\ns t a k e h o l d e r s\nt o\nt r a n s l a t e\nf u n c t i o n a l\na n d\nb u s i n e s s\ns p e c i ﬁ c a t i o n s\ni n t o\ns e a m l e s s\nu s e r\ne x p e r i e n c e s .\n●\nI n\na d d i t i o n ,\nI\nh a v e\na\nw o r k i n g\nk n o w l e d g e\no f\nA W S\ns e r v i c e s ,\ni n c l u d i n g\nA W S\nC l o u d\nD e v e l o p m e n t\nK i t\n( C D K ) ,\nL a m

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
    separators=["\n\n","\n","."," "] 
)

In [7]:
splits = text_splitter.split_documents(docs)

In [8]:
len(splits)

26

In [10]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()

In [11]:
from langchain_community.vectorstores import FAISS, InMemoryVectorStore

db = FAISS.from_documents(splits, embedding)

print(db.index.ntotal)

26


In [12]:
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=HuggingFaceEmbeddings()
)

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever()
)

In [14]:
query = """
We are looking for a software engineer with experience in ReactJS, web development,
and cloud platforms like AWS or Azure. The candidate should have excellent problem-solving skills.
"""

In [17]:
result = qa_chain({"query": query})

In [16]:
print(result["result"])

Based on the provided information, it seems that Gautam Naik may be a suitable candidate for the position. He has:

1. Experience as a Senior Software Engineer with over 8 years of experience in developing front-end systems and responsive websites using React JS, Next JS, TypeScript, Redux, JavaScript, ES6, and CSS.
2. Proficiency in cloud platforms, as he has experience with AWS (Amazon Web Services), including AWS Cloud Development Kit (CDK), Lambda, Serverless Apps, DynamoDB, and SQL.
3. Strong problem-solving skills, which is a requirement for the position.

However, it's worth noting that the job description also mentions Azure, but Gautam's experience is specifically with AWS. If Azure is a requirement, it's unclear if Gautam has experience with it.


In [25]:
db.save_local("cv_db")

In [20]:
new_db = FAISS.load_local("cv_db", embedding, allow_dangerous_deserialization=True)